# Data Science in India - part 2
----
----
In Part 1 we looked at egnder demographics to see if there are any gender sterotypes that still exist in the data science community our analysis showed that there wasn't and things were all similar accross both the genders now in the part 2 we will looking at the industry like size of the company , how much its investing in the data science, what are the types of jobs available how education effects these jobs and what are trends in Data science enthusiasts i-e the part of the community which is more towards learning

 ### <b>Table of Contents </b>
 * [Part 1 : Studying the Gender Demographics](https://www.kaggle.com/zainahmad/data-science-in-india-part-1) <br> 
 * [Part 2 : Industry , Jobs and Education](#part_2)<br>
     2.1 [Jobs and Education](#part_2a)<br> 
     2.2 [How good is the pay](#part_2b)<br>
     2.3 [Company size and number of data scientist](#part_2c)<br>
     2.4 [How does experience helps](#part_2d)<br>
     

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as px
from plotly.subplots import make_subplots
#import chart_studio.plotly as py
import plotly.graph_objects as go
import warnings
#from locale import atof
import math
warnings.filterwarnings('ignore')

In [ ]:
main_2020 = pd.read_csv('../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv')
main_2019 = pd.read_csv('../input/kaggle-survey-2019/multiple_choice_responses.csv')
main_2018 = pd.read_csv('../input/kaggle-survey-2018/multipleChoiceResponses.csv')
main_2017 = pd.read_csv('../input/kaggle-survey-2017/multipleChoiceResponses.csv' , engine='python')

In [ ]:
# for cleaning and extracting the questions
def get_ques_and_clean_df(df):
    cols = df.columns
    names = df.iloc[0,:]
    questions = {a : b  for a, b in zip(cols , names)}
    df.drop([0] , inplace = True)
    df.reset_index(inplace=True)
    df.drop(labels='index',axis=1, inplace=True)
    return questions

# for printing the questions
def print_questions(questions):
    print(f'total number of questions  = {len(questions)}')
    for i in questions.keys():
        print(f'{i} : {questions[i]} \n')

In [ ]:
questions_2018 = get_ques_and_clean_df(main_2018)
questions_2019 = get_ques_and_clean_df(main_2019)
questions_2020 = get_ques_and_clean_df(main_2020)
questions_2017 = {i : v for i , v in enumerate(main_2017.columns)}

In [ ]:
print_questions(questions_2017)

In [ ]:
print_questions(questions_2018)

In [ ]:
print_questions(questions_2019)

In [ ]:
print_questions(questions_2020)

In [ ]:
India_2017 = main_2017.loc[main_2017['Country']=='India']
India_2018 = main_2018.loc[main_2018['Q3']=='India']
India_2019 = main_2019.loc[main_2019['Q3']=='India']
India_2020 = main_2020.loc[main_2020['Q3']=='India']

In [ ]:
India_2020['Q2'] = India_2020['Q2'].replace('Man' , 'Male')
India_2020['Q2'] = India_2020['Q2'].replace('Woman' , 'Female')

<a id='part_2'></a>
## Part 2 - Industry , Jobs and Eduaction
---
we will start from looking at the different types of Jobs how it goes with the education what are the compensations for each of them. how much experience they require etc.



<a id='part_2a'></a>
- ### <u>Jobs and Education</u>

In [ ]:
colors_list = ["15bee0","063732","dd1d57","e0d129","2e7a15","ffa10a","21ff6b","8d0101","8c1896","8bc502"]
colors_list += ["1954b3","3c2a2a","f20a0a","987f34","e17333","2cb579","a9a4e0","e68275"]
colors_list = ['#'+c for c in colors_list]

In [ ]:
Jobs_2020 = India_2020['Q5'].value_counts()
Ed_2020 = India_2020['Q4'].value_counts()

In [ ]:
# a function to generate custom pivot table for cross referencing between two categorical variables
def gen_custom_pivot(df , label1, label2):
    count_1 = df[label1].value_counts()
    #count_2 = df[label2].value_counts()
    df_main = df.loc[df[label1]==count_1.index[0]]
    df_main = pd.DataFrame(df_main[label2].value_counts())
    df_main.rename(columns={label2:count_1.index[0]} , inplace=True)
    for job in count_1.index[1:]:
        df_s = India_2020.loc[India_2020[label1] == job]
        df_main = pd.concat([df_main , pd.DataFrame(df_s[label2].value_counts())] , axis=1)
        df_main.rename(columns={label2:job} , inplace=True)
    df_main.fillna(0 , inplace=True)
    return df_main

In [ ]:
Job_x_Ed = gen_custom_pivot(India_2020 , 'Q5' , 'Q4')
Job_x_Ed

In [ ]:
def plot_dot_plot(df, fig , row):
    colors = colors_list[:len(df.index)]
    colors_dict = {i:c for i,c in zip(df.index , colors)}
    for cols in df.index:
        fig.add_trace(go.Scatter(x=df.loc[cols] , y= df.columns , name = cols , mode='markers' , marker=dict(color=colors_dict[cols] , size=7)) , row=row , col=1)
    fig.update_xaxes(title='Education' , row=row , col=1)
    fig.update_yaxes(title='Job Title' , row=row , col=1)

In [ ]:
fig = make_subplots(rows=4 , cols=2, row_heights = [0.32,0.18,0.5,0], vertical_spacing=0.08,
                   specs = [[{'type':'bar' , 'colspan':2}, None],
                           [{'type':'bar' , 'colspan':2},None],
                           [{'type':'scatter', 'colspan':2 , 'rowspan':2}, None],
                           [None , None]],
                   subplot_titles = ('Distribution of Jobs' , 'Distribution od Education' , 'Jobs against Education'))
fig.add_trace(go.Bar(y= Jobs_2020.index , x= Jobs_2020.values , orientation='h', showlegend=False), row=1, col=1)
fig.add_trace(go.Bar(y= Ed_2020.index , x = Ed_2020.values , orientation='h',showlegend=False), row=2,col=1 )
plot_dot_plot(Job_x_Ed , fig , 3)
fig.update_layout(template='seaborn',height=1100 , width=1000,
                  legend=dict(yanchor='bottom' , y=-0.25 , xanchor='left', x=0.01),
                 title = dict(text='FIGURE 1 : <br>Distribtuion of Education accross Job Titles in Data science sector in India',
                             xanchor='left' , x=0.05))
fig.show()

from the above figure we can see that Bachelor's degree is dominating in almost every job title. for SDE Bachelor's wins with a good margin , for Data scientist Master's take over Bachelor's with a slight margin , In Data Analyst its Bachelor's , ML engineer its Bachelor's. Thus what was concieved for Data Science and Ml domain that you need Master's or Doctor's to work in the Data science field is wrong ans so we can say *THE MYTH IS BURSTED* !!

<a id = 'part_2b'></a>
- ### <u> How Good is the pay</u>

now lets explore what factors effect the pay
1.      pay accorss Job titles

In [ ]:
def decoma(x):
    if type(x) == str:
        symbols = [',' ,'$' , '<' , ">" , '+']
        for sym  in symbols:
            x = x.replace(sym , '')
        if len(x) <= 3:
            x += '000'
    return x

def get_sorted_index(index):
    # preparing bins and labels for binning
    labels = index.copy()
    bins = [a.split('-')[0] for a in labels if type(a) == str ]
    bins = [int(decoma(x)) for x in bins]
    bins.sort()
    # list comprehension to sort the string intervals in correct order using age_bins list
    bin_labels = [c for a in bins for c in labels if int(decoma(c.split('-')[0])) == a]
    return bin_labels

In [ ]:
pay_x_Job = gen_custom_pivot(India_2020 , 'Q5' ,'Q24')
pay_x_Job = pay_x_Job.reindex(get_sorted_index(pay_x_Job.index))
pay_x_Job.drop(columns=['Student', 'Currently not employed'],axis=1, inplace=True)
pay_x_Job

In [ ]:
subplot_titles = (list(pay_x_Job.columns))
subplot_titles.append(' ')

In [ ]:
fig = make_subplots(rows = len(subplot_titles) , cols=2, vertical_spacing=0.012,
                   specs=([[{'type':'bar', 'colspan':2}, None]]*(len(subplot_titles)-1))+[[{'type':'indicator', 'colspan':2}, None]],
                   subplot_titles = subplot_titles)
for i , v in enumerate(pay_x_Job.columns):
    fig.add_trace(go.Bar(x=pay_x_Job.index , y=pay_x_Job[v], showlegend=False), row=i+1, col=1)
    if i+1 != len(pay_x_Job.columns):
        fig.update_xaxes(showticklabels=False , row=i+1 , col=1)
for i in fig['layout']['annotations']:
    i['font'] = dict(size=12 )
    i['xanchor'] = 'left'
    i['x']=0.02
fig.update_layout(height=1800 , title_text='FIGURE 2 : <br>Salary of DS and ML professionals in India accross different job roles')
fig.show()

2. pay accross education

In [ ]:
Ed_x_Job = gen_custom_pivot(India_2020 , 'Q4' ,'Q24')
Ed_x_Job = Ed_x_Job.reindex(get_sorted_index(Ed_x_Job.index))
Ed_x_Job

In [ ]:
subplot_titles = list(Ed_x_Job.columns)
subplot_titles = [str(i)+'. '+c for i,c in zip(range(1,len(subplot_titles)+1), subplot_titles)]

In [ ]:
fig = make_subplots(rows = len(subplot_titles) , cols=2, vertical_spacing=0.025,
                   specs=([[{'type':'bar', 'colspan':2}, None]]*(len(subplot_titles))),
                   subplot_titles = subplot_titles)
for i , v in enumerate(Ed_x_Job.columns):
    fig.add_trace(go.Bar(x=Ed_x_Job.index , y=Ed_x_Job[v], showlegend=False), row=i+1, col=1)
    if i+1 != len(Ed_x_Job.columns):
        fig.update_xaxes(showticklabels=False , row=i+1 , col=1)
for i in fig['layout']['annotations']:
    i['font'] = dict(size=12 )
    i['xanchor'] = 'left'
    i['x']=0.02
fig.update_layout(height=900 , title_text='FIGURE 3 : <br>Salary of DS and ML professionals in India by level of education')
fig.show()

we can see that most of the people are getting paid under USD 1000 anually how ever when we look at the higher pay i-e above 10,000 USD than its higher degrees which are winning which is obvious because they come with good ammount of experience.<br>

But in General case if we look at the distribution we can conclude that Bachelor's is enough to start a professional career.<br>

another important insight we get here is that we can see there are people who are working in this field who don't have a professional degree (graph 3 and 7 in Figure 3). which is a good sign, it shows that one can make a good career with shear skill and talent even if he/she don't have a proper education, all you need is to work hard

<a id='part_2c'> </a>
- ### Company size and number of data scientist

In [ ]:
company_size = India_2020[['Q20' , 'Q21']]
company_size.shape

In [ ]:
company_size.isna().sum()

In [ ]:
company_size.dropna(inplace=True)
company_size.head()

In [ ]:
fig= make_subplots(rows=2 , cols=1 , subplot_titles= ('Distribtuion of Company size' , 'Number of Data scientists in a company'))
df = company_size['Q20'].value_counts()
fig.add_trace(go.Bar(x = df.values , y = df.index , showlegend=False , orientation = 'h'),row=1 , col=1)
df2 = company_size['Q21'].value_counts()
df2 = df2.reindex(get_sorted_index(df2.index))
fig.add_trace(go.Bar(x= df2.values , y=df2.index , showlegend=False , orientation='h'), row=2,col=1)
fig.update_layout(title_text = 'FIGURE 4 : <br>Company size and number of Data scientists in India')
fig.show()

<a id='#part_2d'></a>
- ### How does experience helps

In [ ]:
ML_and_pro = India_2020[['Q15' , 'Q6']]
print(ML_and_pro['Q15'].unique())
print(ML_and_pro['Q6'].unique())

Distribtuion of Experience

In [ ]:
fig=  make_subplots(rows=2 , cols=1, subplot_titles=('Experience in Coding' , 'Experience in Machine learning methods'))
df = India_2020['Q6'].value_counts()
fig.add_trace(go.Bar(x=df.values , y=df.index , showlegend=False , orientation='h'), row=1, col=1)
df2 = India_2020['Q15'].value_counts()
fig.add_trace(go.Bar(x=df2.values , y=df2.index , showlegend=False , orientation='h'), row=2, col=1)
fig.update_layout(title_text='FIGURE 5 : <br>Distribtuion of Work experience in ML and coding in Indian DS community', height=600)
fig.show()

How expeience in these two aspects efect the salary

In [ ]:
sal_and_code = gen_custom_pivot(India_2020 , 'Q6' , 'Q24')
sal_and_ml = gen_custom_pivot(India_2020 , 'Q15' ,'Q24')

In [ ]:
colors= sns.color_palette('inferno' , 25)
colors = ['rgb'+str(c) for c in colors]
colors_dict = {i:c for i,c in zip(sal_and_code.index , colors)}
colors_dict

In [ ]:
def plot_bars(df, figure , row , colors_dict :dict , showlegend=False):
    for i in df.index:
        fig.add_trace(go.Bar(y=df.columns , x=df.loc[i] ,orientation='h', name=i , marker = dict(color=colors_dict[i]), showlegend=showlegend), row=row, col=1)


In [ ]:
fig = make_subplots(rows=2 , cols=1 , subplot_titles = ('Coding' , 'ML'))
plot_bars(sal_and_code , fig , 1 , colors_dict , showlegend=True)
plot_bars(sal_and_ml , fig , 2, colors_dict)
fig.update_layout(title_text = 'FIGURE 6 : <br>Effect of Experience on salary' , barmode='stack', height=600 , width=1000)
fig.show()

In [ ]:
# group_count = pd.DataFrame(India_2020.groupby(['Q6' , 'Q24'])['Q24'].count())
# group_count = group_count.rename(columns={'Q24':'Values'})
# group_count['Q6'] = group_count.index.get_level_values(0)
# group_count['Q24'] = group_count.index.get_level_values(1)
# group_count = group_count[['Q6' , 'Q24' , 'Values']]
# group_count.reset_index(drop=True , inplace=True)
# group_count